## Importing dataset into folder

In [ ]:
!git clone https://github.com/GoldenCorgi/CZ2001.git
!unzip CZ2001/graph_dataset.zip

# Problem Statement

1. Problem
You are given an undirected unweighted graph G, which represents a city’s road
network with n nodes being intersections/endpoints and m edges being roads. Among
the n nodes, h of them are hospitals and you are interested in finding, for each node,
the distance (i.e., the number of edges in the shortest path) from each node to the
nearest hospital. h could take any value from 1 to n.  
(a) Design an algorithm for computing the distance from each node in G to its
nearest hospital. Output the distance and the shortest path for each node to a
file.  
(b) Design an algorithm to complete the task (a) but its time complexity should not
depend on the total number of hospitals h. You could skip (b) if your algorithm
in (a) already satisfies this complexity constraint.  
(c) In some circumstances (e.g., big disasters, fires, etc.), having the distance from
each node to the nearest hospital is not good enough. Instead, we are
interested in finding the distances to top-2 nearest hospitals from each node.
Revise your algorithm in (b) to accommodate this new requirement. If revision
is not possible, you are free to design a new algorithm.  
(d) Propose an algorithm that works generally for computing the distances from
each node to top-k nearest hospitals for an input value of k.


## Imports of libraries and packages, and graph datasets

In [19]:
import time
from tqdm.notebook import tqdm
import os
import random
import sys
import gc


In [20]:
for root, dirs, files in os.walk("graph_dataset"):
    for filename in files:
        print("graph_dataset/" + filename)

graph_dataset/Simple_pairwise-London_tube_map.txt
graph_dataset/Simple_pairwise-Blackhole.txt
graph_dataset/hospitals.txt
graph_dataset/roadNet-CA.txt


In [21]:
def import_hospitals(filename):
    with open(filename) as f:
        glist = (f.readlines())
        glist = [int(x.strip()) for x in tqdm(glist, desc='Parsing Text Files') if x.strip()[0] != "#"]
    return glist


In [22]:
def import_random_graph():
    with open("graph_dataset/Simple_pairwise-Blackhole.txt") as f:
        glist = (f.readlines())
        glist = [x.replace("\t",",").strip() for x in glist if x.strip()[0] != "#"]
        graphRoadList = [[int(y) for y in x.split(",")] for x in glist]
        del glist
    return graphRoadList

In [23]:
def import_graph_network(filename):
    with open(filename) as f:
        glist = (f.readlines())

        glist = [x.replace("\t",",").strip() for x in tqdm(glist, desc='Parsing Text Files') if x.strip()[0] != "#"]
        graphRoadList = [[int(y) for y in x.split(",")] for x in tqdm(glist, desc='Converting to Integers')]
        del glist
    return graphRoadList

In [24]:
def import_roadnet(n=100000):
    with open("graph_dataset/roadNet-CA.txt") as f:
        glist = (f.readlines())
        if n != 0:

            glist = glist[:n]

        glist = [x.replace("\t",",").strip() for x in tqdm(glist, desc='Parsing Text Files') if x.strip()[0] != "#"]
        graphRoadList = [[int(y) for y in x.split(",")] for x in tqdm(glist, desc='Converting to Integers')]
        del glist
    return graphRoadList
# graphRoadList = import_roadnet(n=100000) # 0 for all

## Definition of functions and data structures

In [25]:
class Node():
    # self.idx = int(node_number)
    # self.connected_nodes = [Node(),Node()]
    # self.nearest_hospitals = [(distance,path),(distance,path)]

    def __init__(self,idx):
        self.idx = idx
        self.connected_nodes = set()
        self.nearest_hospitals = []
        self.visited_hospitals = list()

def ConvertToGraphNodeStructure(graphRoadList,show_tqdm = True):
    GraphNodes = {}
    if show_tqdm:
        for (node_start,node_next) in tqdm(graphRoadList, desc='Converting to Graph Node Structure'):
            if node_next not in GraphNodes:
                GraphNodes[node_next] = Node(node_next)
            if node_start not in GraphNodes:
                GraphNodes[node_start] = Node(node_start)
            GraphNodes[node_start].connected_nodes.add(GraphNodes[node_next])
            GraphNodes[node_next].connected_nodes.add(GraphNodes[node_start])
        return GraphNodes
    else:
        for (node_start,node_next) in graphRoadList:
            if node_next not in GraphNodes:
                GraphNodes[node_next] = Node(node_next)
            if node_start not in GraphNodes:
                GraphNodes[node_start] = Node(node_start)
            GraphNodes[node_start].connected_nodes.add(GraphNodes[node_next])
            GraphNodes[node_next].connected_nodes.add(GraphNodes[node_start])
        return GraphNodes



In [26]:
def SimultaneousSplitBFS(GraphNodes,k=1,h=10,show_tqdm=True, refresh=True):
    start = time.time()
    VISITED = {x:0 for x in list(GraphNodes.keys())}
    LISTQUEUE = []
    if refresh:
        for i in GraphNodes:
            GraphNodes[i].nearest_hospitals = []
            GraphNodes[i].visited_hospitals = list()

    if type(h) == int:
        HOSPITALS = random.choices(list(GraphNodes.keys()),k=h)
    else: # input hospitals list
        HOSPITALS = h
    # Initialise Hospitals
    for i in HOSPITALS:

        # FIFOQUEUE.put((i,[]))
        if i not in GraphNodes:
            print(i, " Not in Graph")
            continue
        GraphNodes[i].visited_hospitals.append(i)
        LISTQUEUE.append((i,[]))

        VISITED[i] += 1
    n = 0 


    # k = 1
    if show_tqdm:
        t = tqdm(total=k*len(GraphNodes), desc='Running Simultaneous Split BFS') # Initialise

    while LISTQUEUE:

        n += 1
        if (n % 100000 == 0):
            if show_tqdm:
                t.update(100000)
        node_to_visit, path_to_hospital = LISTQUEUE.pop(0)
        new_path = path_to_hospital + [node_to_visit]
        if k > 1:
            GraphNodes[node_to_visit].nearest_hospitals.append((len(path_to_hospital),new_path[0]))
        else:
            GraphNodes[node_to_visit].nearest_hospitals.append((len(path_to_hospital),path_to_hospital))
        
        new_nodes_to_visit = GraphNodes[node_to_visit].connected_nodes
        

        for nodes in new_nodes_to_visit:
            if VISITED[nodes.idx] < k:
                if new_path[0] not in GraphNodes[nodes.idx].visited_hospitals:
                    LISTQUEUE.append((nodes.idx,new_path))
                    GraphNodes[nodes.idx].visited_hospitals.append(new_path[0])
                    VISITED[nodes.idx] += 1
    
    
    end = time.time()
    if show_tqdm:
        t.update(k*len(GraphNodes)%100000)
        t.close()
    return GraphNodes

In [27]:
def get_to_text_file(GraphNodes, k=1,save_file=False):
    if k>1:
        paths = False
    else:
        paths = True
    n = 0
    if save_file:
        with open(save_file, "a") as myfile:
            for nodes in tqdm(GraphNodes.values(), desc='Saving Results to Text File'):
                tmpstring = ""
                tmpstring += ("Node: " + str(nodes.idx) + "\n")
                # tmpstring +=("Connected Nodes: " + str([x.idx for x in nodes.connected_nodes])+ "\n")
                tmpstring +=(f"Has Nearest {len(nodes.nearest_hospitals)}/{k} Hosp: "+ "\n")
                ik = 1
                for y,x in nodes.nearest_hospitals:
                    if paths:
                        if not x: # hospital node
                            tmpstring +=(f"\t[{ik}] Hosp Node: \t"+ str(nodes.idx)+ "\n")
                        else:
                            tmpstring +=(f"\t[{ik}] Hosp Node: \t"+ str(x[0])+ "\n")
                        tmpstring +=(f"\t[{ik}] Dist: \t"+str(y)+ "\n")
                        tmpstring +=(f"\t[{ik}] Path: \t" + str(x)+ "\n")
                    else:
                        tmpstring +=(f"\t[{ik}] Hosp Node: \t"+ str(x)+ "\n")
                        tmpstring +=(f"\t[{ik}] Dist: \t"+str(y)+ "\n")
                    ik += 1
                myfile.write(tmpstring + "\n")
          
    else:
        for nodes in (GraphNodes.values()):
            print("Node: " + str(nodes.idx))
            # print("Connected Nodes: " + str(nodes.connected_nodes))
            print("Connected Nodes: " + str([x.idx for x in nodes.connected_nodes]))
            print(f"Has Nearest {len(nodes.nearest_hospitals)}/{k} Hospital(s): ")
            ik = 1
            for y,x in nodes.nearest_hospitals:
                if paths:
                    print(f"\t[{ik}] Hosp Node: \t"+ str(x[0]))
                    print(f"\t[{ik}] Distance: \t"+str(y))
                    print(f"\t[{ik}] Path: \t" + str(x))
                else:
                    print(f"\t[{ik}] Hosp Node: \t"+ str(x))
                    print(f"\t[{ik}] Distance: \t"+str(y))
                ik += 1
            print()
            n += 1
            if n == 2:
                break

## Demo Outputs for Presentation

In [28]:
graphRoadList = import_random_graph()
GraphNodes = ConvertToGraphNodeStructure(graphRoadList,show_tqdm=False)
print("SS-BFS for Part A/B")
OutputNodes1 = SimultaneousSplitBFS(GraphNodes,k=1,h=20)
get_to_text_file(OutputNodes1, k=1,save_file=False)


SS-BFS for Part A/B



Node: 1
Connected Nodes: [2, 842, 212, 422, 1, 632]
Has Nearest 1/1 Hospital(s): 
	[1] Hosp Node: 	10
	[1] Distance: 	4
	[1] Path: 	[10, 6, 4, 2]

Node: 2
Connected Nodes: [844, 2, 842, 4, 1, 3, 212]
Has Nearest 1/1 Hospital(s): 
	[1] Hosp Node: 	10
	[1] Distance: 	3
	[1] Path: 	[10, 6, 4]



In [29]:
print("SS-BFS for Part C (k=2)")

OutputNodes2 = SimultaneousSplitBFS(GraphNodes,k=2,h=20)
get_to_text_file(OutputNodes2, k=2,save_file=False)


SS-BFS for Part C (k=2)



Node: 1
Connected Nodes: [2, 842, 212, 422, 1, 632]
Has Nearest 2/2 Hospital(s): 
	[1] Hosp Node: 	234
	[1] Distance: 	7
	[2] Hosp Node: 	94
	[2] Distance: 	14

Node: 2
Connected Nodes: [844, 2, 842, 4, 1, 3, 212]
Has Nearest 2/2 Hospital(s): 
	[1] Hosp Node: 	234
	[1] Distance: 	7
	[2] Hosp Node: 	94
	[2] Distance: 	13



In [30]:
print("SS-BFS for Part D (k=3)")

OutputNodes3 = SimultaneousSplitBFS(GraphNodes,k=3,h=20)
get_to_text_file(OutputNodes3, k=3,save_file=False)

SS-BFS for Part D (k=3)



Node: 1
Connected Nodes: [2, 842, 212, 422, 1, 632]
Has Nearest 3/3 Hospital(s): 
	[1] Hosp Node: 	11
	[1] Distance: 	4
	[2] Hosp Node: 	667
	[2] Distance: 	8
	[3] Hosp Node: 	56
	[3] Distance: 	10

Node: 2
Connected Nodes: [844, 2, 842, 4, 1, 3, 212]
Has Nearest 3/3 Hospital(s): 
	[1] Hosp Node: 	11
	[1] Distance: 	3
	[2] Hosp Node: 	667
	[2] Distance: 	8
	[3] Hosp Node: 	56
	[3] Distance: 	9



## Getting output files for submission

In [31]:
graphRoadList = import_random_graph()
GraphNodes = ConvertToGraphNodeStructure(graphRoadList)
OutputNodes1 = SimultaneousSplitBFS(GraphNodes,k=1,h=20)
get_to_text_file(OutputNodes1, k=1,save_file="random_graph_a.txt")
OutputNodes2 = SimultaneousSplitBFS(GraphNodes,k=2,h=20)
get_to_text_file(OutputNodes2, k=2,save_file="random_graph_c.txt")
OutputNodes3 = SimultaneousSplitBFS(GraphNodes,k=3,h=20)
get_to_text_file(OutputNodes3, k=3,save_file="random_graph_d.txt")

## Using personal graph network and personal hospital list, run a SS-BFS, altering k=1 for any values of k

In [32]:
file1 = "graph_network.txt"
file2 = "hospitals.txt"

In [33]:
# graphRoadList = import_graph_network(file1)
# hospital_list = import_hospitals(file2)
# GraphNodes = ConvertToGraphNodeStructure(graphRoadList)
# OutputNodes = SimultaneousSplitBFS(GraphNodes,k=1,h=hospital_list)
# get_to_text_file(OutputNodes, k=1,save_file="output.txt")

## Using California Network, run a SS-BFS of all the nodes using randomly chosen hospitals h=350.

In [17]:
graphRoadList = import_roadnet(n=0) # 0 for all
GraphNodes = ConvertToGraphNodeStructure(graphRoadList)
OutputNodes = SimultaneousSplitBFS(GraphNodes,k=1,h=350) # h is randomly chosen if number, or chosen directly if in a hospital_list
get_to_text_file(OutputNodes, k=1,save_file="output.txt")

In [34]:
!zip -9 output.zip output.txt

updating: output.txt (deflated 96%)
updating: output.txt (deflated 96%)
